# Zeitzonen exportieren

In [1]:
from pathlib import Path
import geopandas as gpd
import json
import os
import consts

## Create 15° Lines

In [2]:
features = []

for i in range(0, 24):

    lon = (i * 15) - 180 + 7.5

    features.append({
        'type': 'Feature',
        'properties': {
            'timezone': i,
        },
        'geometry': {
            'type': 'LineString',
            'coordinates': [
                [lon, -90],
                [lon, 90],
            ]
        }
    })


json.dump({
    'type': 'FeatureCollection',
    'features': features
}, open(consts.PATH_GEOJSON_TIMEZONES, 'w', encoding='utf-8'), ensure_ascii=False)

## Add Timezones to Country shapes

In [3]:
gdf_countries_raw = gpd.read_file(Path('../data/countries-europe.gpkg'))

In [4]:
gdf_countries = gdf_countries_raw.copy()

# Set Timezone
gdf_countries['timezone_is'] = 1
gdf_countries.loc[gdf_countries['ISO_A2_EH'].isin(['PT', 'GB', 'IE', 'IS', 'IM', 'FO']), 'timezone_is'] = 0
gdf_countries.loc[gdf_countries['ISO_A2_EH'].isin(['FI', 'EE', 'LT', 'LV', 'RO', 'BG', 'GR', 'MD', 'UA']), 'timezone_is'] = 2
gdf_countries.loc[gdf_countries['ISO_A2_EH'].isin(['RU', 'BY', 'TK']), 'timezone_is'] = 3

# Timezone should
gdf_countries.loc[gdf_countries['ISO_A2_EH'].isin(['IS', 'IE', 'PT']), 'timezone_should'] = -1
gdf_countries.loc[gdf_countries['ISO_A2_EH'].isin(['ES', 'GB', 'FR', 'BE', 'NL', 'LU', 'AD', 'JE', 'GG', 'IM', 'FO']), 'timezone_should'] = 0
gdf_countries.loc[gdf_countries['ISO_A2_EH'].isin([
    'DE', 'CH', 'IT', 'AT', 'PL', 'CZ', 'SI', 'HR', 'BA', 'ME', 'AL', 'MK', 'XK', 'RS', 'HU', 'SK', 'NO', 'SE', 'DK',
     'MC', 'LI', 'MC', 'SM', 'MT'
]), 'timezone_should'] = 1
gdf_countries.loc[gdf_countries['ISO_A2_EH'].isin(['FI', 'EE', 'LT', 'LV', 'UA', 'RO', 'BG', 'GR', 'MD', 'BY', 'AX']), 'timezone_should'] = 2

# Drop Countries
gdf_countries = gdf_countries[~gdf_countries['ISO_A2_EH'].isin(['RU', 'TK'])]

gdf_countries = gdf_countries[['geometry', 'ISO_A2_EH', 'timezone_is', 'timezone_should']]

gdf_countries.to_file(consts.PATH_GEOJSON_COUNTRIES)

## Merge everything

In [3]:
# Run Tippecanoe
os.system(f"tippecanoe -zg -o {consts.PATH_WEBEXPORT / 'zeitumstellung.mbtiles'} --drop-densest-as-needed {consts.PATH_GEOJSON_COUNTRIES} {consts.PATH_GEOJSON_EXPORT} {consts.PATH_GEOJSON_TIMEZONES} {consts.PATH_GEOJSON_HOURLY} --force")

For layer 0, using name "countries"
For layer 1, using name "grid"
For layer 2, using name "timezonelines"
For layer 3, using name "gridhourly"
12568 features, 3727477 bytes of geometry, 527508 bytes of string pool
Choosing a maxzoom of -z1 for features typically 162148 feet (49423 meters) apart, and at least 54232 feet (16530 meters) apart
Choosing a maxzoom of -z4 for resolution of about 21900 feet (6675 meters) within features
tile 0/0/0 size is 619247 with detail 12, >500000    
Going to try keeping the sparsest 72.67% of the features to make it fit
tile 1/1/0 size is 593519 with detail 12, >500000    
Going to try keeping the sparsest 75.82% of the features to make it fit
tile 2/2/1 size is 570625 with detail 12, >500000    
Going to try keeping the sparsest 78.86% of the features to make it fit
tile 3/4/2 size is 554232 with detail 12, >500000    
Going to try keeping the sparsest 81.19% of the features to make it fit
  99.9%  4/6/5    


0